In [1]:
from baseline_cnn import *
from baseline_cnn import BasicCNN
from copy import deepcopy


# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
#transform = transforms.Compose(______) #starter code
transform = transforms.Compose(transforms.ToTensor())


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = BasicCNN()
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = ______ #TODO - loss criteria are defined in the torch.nn package
criterion = torch.nn.BCELoss(size_average= True)
#TODO: change loss function to weighted loss

#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
#optimizer = ______ #TODO - optimizers are defined in the torch.optim package #this is the start code line
optimizer = torch.optim.adam

CUDA NOT supported


FileNotFoundError: File b'/datasets/ChestXray-NIHCC/Data_Entry_2017.csv' does not exist

In [ ]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
valid_loss = []
best_model = model

# Begin training procedure
for epoch in range(num_epochs):

    N = 50
    N_minibatch_loss = 0.0    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):

        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement cross-validation
        minibatch_count % N == 0:
            
            (images, labels) in enumerate(val_loader, 0):
                outputs = model(images)
                loss = criterion(outputs, labels)
                #checks if model is best model so far
                if (loss < min(valid_loss)):
                    model = deepcopy(model)
                valid_loss.append(loss)
            
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

In [ ]:
#Testing Block (looking to find all the statistics and confusion matrix)
#test based on best_model variable
(images, labels) in enumerate(test_loader, 0):
    outputs = model(images)
    #complete tests (all 4 scores) (for each class)
    """
    Idea:
    1. Iterate through the output vs. label
    2. Place each correct/incorrect into piles of: true negatives, false negatives, true positives, and false positives
        - note: different pile for each class
        - note: could make a matrix of size (labels + 1 x 4) where the i input is the true label and the j input is the correct category
    3. Use these piles to calculate score for each class
    """
    #complete confusion matrix
    """
    Idea:
    1. Create matrix of size (labels x labels)
    2. Iterate through output vs. labels
    3. increase value by 1 in each corresponding i,j spot
    4. Divide by average across each row afterwards.
    """

In [ ]:
#Custom Loss Function
#Based on a weights on Negatives and Positives
def weighted_Loss_Function(output, target, w_n = None, w_p = None):
    
    #puts a range on the outputs to avoid a nan error
    output = output.clamp(min=1e-7, max=1-1e-7)
    
    target = target.float()
    if (w_n == None & w_p == None):
        #default loss function if no weight parameters given
        loss = (-1) * target * torch.log(output) - (1 - target) * torch.log(1 - output)
    else:
        loss = (-1) * w_n * (target * torch.log(output)) - w_p * ((1 - target) * torch.log(1 - output))
    
    return torch.sum(loss)

In [ ]:
#need to find the w_n and w_p parameters. 
"""
w_n = (|n| + |p|) / |n| 

w_p = (|n| + |p|) / |p| 

Goal: find the number of negative and positive outputs in the outputs.
Positives = they have a disease
Negative = they do not have a disease

Procedure: loop through the outputs hot-coded arrays. If there is a 1, then it counts as positive
"""